In [1]:
import starepandas
import geopandas
import pickle
import pystare
import matplotlib.pyplot as plt
import datetime
import numpy

In [2]:
# We set level to 15 because that appears to be matching VNP02 nadir resolution 
level = 15

In [3]:
caribbean = geopandas.read_file('study_area_fao_clean_210326.gpkg')

In [ ]:
caribbean = starepandas.STAREDataFrame(caribbean, add_stare=True, level=level, add_trixels=True, n_workers=60)

with open('caribbean_staredf.pickle', 'wb') as f:
    pickle.dump(caribbean, f)

In [ ]:
# Or load it from pickle
with open('caribbean_staredf.pickle', 'rb') as f:
    caribbean = pickle.load(f)

In [ ]:
sids = list(caribbean['stare'])
numpy.concatenate(sids).size

# STARE Cover for the whole ROI

In [ ]:
start = datetime.datetime.now()
cover_sids = caribbean.stare_dissolve(n_workers=10)
print(datetime.datetime.now() - start)

In [ ]:
len(sids)

In [ ]:
with open('caribbean_sids_cover.pickle', 'wb') as f:
    pickle.dump(sids, f)

In [ ]:
with open('caribbean_sids_cover.pickle', 'rb') as f:
    cover_sids = pickle.load(f)

# Making countrywise DF

## Simplify -> Dissolve -> STARE lookup

In [ ]:
geom_simple = caribbean.simplify(0.001, preserve_topology=True)
caribbean['geom_simple'] = geom_simple

n_geom = 0
n_simple = 0

for index, row in caribbean.iterrows():
    for p in row.geometry:
        n_geom += len(p.exterior.coords)    
        
    if row.geom_simple.geom_type =='Polygon':
        n_simple += len(row.geom_simple.exterior.coords)
    else:
        for p in row.geom_simple:
            n_simple += len(p.exterior.coords)

print(n_geom)
print(n_simple)

In [ ]:
caribbean.set_geometry('geom_simple', inplace=True)

In [ ]:
# Just to verify if simplification is OK
fig, ax = plt.subplots(figsize=(3,3), dpi=200)
ax.grid(True)

country = caribbean[caribbean.ADM0_NAME=='Dominica']
country.plot(ax=ax, column='ADM1_CODE')

In [ ]:
# Option a
start = datetime.datetime.now()
countries = caribbean.dissolve(by='ADM0_CODE')
countries1 = starepandas.STAREDataFrame(countries, add_stare=True, level=level, add_trixels=True, n_workers=20)
print(datetime.datetime.now() - start)

In [ ]:
# Option b
start = datetime.datetime.now()
countries2 = caribbean.stare_dissolve(by='ADM0_CODE', geom=False, n_workers=20)
trixels = countries2.trixels(n_workers=20)
countries2.set_trixels(trixels, inplace=True)
print(datetime.datetime.now() - start)

In [ ]:
fig, ax = plt.subplots(figsize=(5,5), dpi=200)
ax.grid(True)

countries1.plot(ax=ax, trixels=True, column='ADM0_NAME', linewidth=0.1)

In [ ]:
fig, ax = plt.subplots(figsize=(5,5), dpi=200)
ax.grid(True)

countries2.plot(ax=ax, trixels=True, column='ADM0_NAME', linewidth=0.1)

In [ ]:
# Simplified STARE cover
start = datetime.datetime.now()
cover_sids = countries1.stare_dissolve(n_workers=10)
print(datetime.datetime.now() - start)

# Test if granule intersects

In [ ]:
path = 'data/'
granule_trunk = 'VNP03DNB'

catalogue = starepandas.folder2catalogue(path=path, 
                                         granule_trunk=granule_trunk,
                                         granule_extension='nc')

In [ ]:
#granule_cover = catalogue.iloc[0]['stare_cover']
intersects = catalogue.stare_intersects(cover_sids)